In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import cPickle as pk
import pandas as pd
import numpy as np
from IPython.display import clear_output

# Munging US CPC to IPC into data Checking Compatibility

## SQL Query used on PATSTAT 2015 Spring

## Importing
[to top](#Table-of-Contents) 

In [2]:
pwd

u'/Users/innovation-sfi/Dropbox/Research/Innovation/EPO/Data_processing/Python'

# CPC

In [3]:
f_all_grants = '../Data_Play/all_grant_only_prior_US_cpc_v1.csv'
USdf = pd.read_csv(f_all_grants, engine='c', dtype={'publn_nr':str,'appln_id':str})

In [4]:
len(USdf)

15150421

# IPC

In [5]:
tot_file = '../Data_Play/all_grant_only_prior_v1.csv'
df = pd.read_csv(tot_file, engine='c', dtype={'publn_nr':str,'appln_id':str})

In [6]:
print len(df)
print len(df.appln_id.unique())
print df.head()
print df.groupby('appln_id').size().value_counts()

45538451
21788931
  publn_nr appln_auth  publn_date  appln_id ipc_class_symbol
0  PP19626         US  2009-01-20  55754593      A01H   5/00
1  PP19627         US  2009-01-20  52899962      A01H   5/00
2  PP19628         US  2009-01-20  55754594      A01H   5/00
3  PP19629         US  2009-01-20  55754595      A01H   5/00
4  PP19630         US  2009-01-20  55318791      A01H   5/00
1      11239797
2       5116710
3       2478017
4       1287590
5        670656
6        378136
7        220937
8        131309
9         81798
10        52379
11        34645
12        23356
13        16673
14        12091
15         8738
16         7084
17         5210
18         3891
19         3112
20         2410
21         1928
22         1582
23         1350
24         1209
25         1010
27          752
26          714
28          597
29          517
30          493
         ...   
247           1
183           1
157           1
177           1
135           1
96            1
99            1
103     

In [7]:
# fraction of patents with only one code
print float(11239797)/21788931

0.515848941832


# EP country codes  

In [8]:
file_ep = '../Data_Play/all_grant_prior_only_EP_cntrycds_v1.csv'
ep_df = pd.read_csv(file_ep, engine='c', dtype={'appln_id':str})

In [9]:
print len(ep_df)
print ep_df
print ep_df.person_ctry_code.value_counts().head()

69793
      appln_auth appln_id person_ctry_code
0             EP   101898               DE
1             EP   101908               DE
2             EP   101909               CH
3             EP   101922               JP
4             EP   101960               DE
5             EP   101962               AT
6             EP   101965               BE
7             EP   101978               DE
8             EP   101980               DE
9             EP   102009               DE
10            EP   102010               DE
11            EP   102027               DE
12            EP   102046               DE
13            EP   102051               DE
14            EP   102067               DE
15            EP   102076               DE
16            EP   102102               CH
17            EP   102104               DE
18            EP   102120               DE
19            EP   102129               CH
20            EP   102129               DE
21            EP   102131               DE
22   

In [10]:
dupes = 0
ss_ep_dupes = ep_df.groupby('appln_id').size().value_counts()
for inx in ss_ep_dupes[1:].index:
    dupes = dupes + (inx * ss_ep_dupes.loc[inx])
print dupes
print ss_ep_dupes
# sum over id that appear more than once, multiplies by their number of appearences, 
# gives number of 'duplicate' patents added

10800
1    58993
2     4673
3      420
4       42
5        4
6        1
dtype: int64


In [11]:
# check if all apps in EP to countrys is in df, and vice versa (
# (can see it must be untrue in one case because of of their lengths differences)
df_EP_unique = len(df[df.appln_auth == 'EP'].drop_duplicates('appln_id'))
df_EP_unique_in_cntry_list =  df[df.appln_auth == 'EP'].drop_duplicates('appln_id').appln_id.isin(ep_df.drop_duplicates('appln_id').appln_id).sum()
print df_EP_unique - df_EP_unique_in_cntry_list
print 'Indicates how many will be lost '
len(ep_df.drop_duplicates('appln_id'))
ep_df.drop_duplicates('appln_id').appln_id.isin(df[df.appln_auth == 'EP'].drop_duplicates('appln_id').appln_id).sum()

4182
Indicates how many will be lost 


64120

In [12]:
df.head()

,publn_nr,appln_auth,publn_date,appln_id,ipc_class_symbol
0,PP19626,US,2009-01-20,55754593,A01H 5/00
1,PP19627,US,2009-01-20,52899962,A01H 5/00
2,PP19628,US,2009-01-20,55754594,A01H 5/00
3,PP19629,US,2009-01-20,55754595,A01H 5/00
4,PP19630,US,2009-01-20,55318791,A01H 5/00


In [13]:
df_ctry_replc = df[df['appln_id'] == None]
print df_ctry_replc

Empty DataFrame
Columns: [publn_nr, appln_auth, publn_date, appln_id, ipc_class_symbol]
Index: []


In [14]:
df_just_ep = df[df['appln_auth'] == 'EP'].copy()
len(df_just_ep)

195966

In [15]:
# Munge in the EP countries, by appending each row corresponding to an appln_id with the appln auth switched
#df_unique_ids = df.appln_id.unique()
for inx in ep_df.index:
    print inx,
    ap_id = ep_df.appln_id.loc[inx]
    #if ap_id in df_unique_ids:
    temp_df = df_just_ep[df_just_ep['appln_id'] == ap_id].copy()
    temp_df['appln_auth'] = ep_df.person_ctry_code.loc[inx]
    df_ctry_replc = df_ctry_replc.append(temp_df)
print df_ctry_replc

          publn_nr appln_auth  publn_date appln_id ipc_class_symbol
3495810   1526079         DE  2008-10-01   101898      B65D   8/02
3495811   1526079         DE  2008-10-01   101898      B65D   8/04
3495812   1526079         DE  2008-10-01   101898      B65D  25/44
3495813   1526079         DE  2008-10-01   101898      B65D  47/06
3495814   1526079         DE  2008-10-01   101898      B65D  51/24
3495815   1526079         DE  2008-10-01   101898      B65D  51/28
2376467   1655548         DE  2008-10-22   101908      F16L   5/00
2376468   1655548         DE  2008-10-22   101908      F24H   9/02
2519528   1610120         CH  2009-09-23   101909      G01N  27/28
6378393   1628467         JP  2009-09-30   101922      B41M   1/14
6378394   1628467         JP  2009-09-30   101922      G06T   1/00
6378395   1628467         JP  2009-09-30   101922      H04N   1/46
6378396   1628467         JP  2009-09-30   101922      H04N   1/60
22021646  1989978         DE  2010-09-01   101960      A47J  

In [16]:
# check no new patents in set
print len(df.append(df_ctry_replc, ignore_index=True).appln_id.unique())
print len(df.appln_id.unique())

21788931
21788931


In [17]:
# append the new EPtoCntry rows onto df
df = df.append(df_ctry_replc, ignore_index=True)

In [18]:
# then filter off any EP patents
df = df[df['appln_auth'] != 'EP']
print 

In [19]:
# see number of lost patents
print 21788931 - len(df.appln_id.unique())
# matches with previous result on lost patents check

4182


In [20]:
USdf_needed = USdf[~USdf.appln_id.isin(df.appln_id)]; print USdf_needed.head()
len(USdf_needed)

  publn_nr appln_auth  publn_date  appln_id cpc_class_symbol
0   718793         US  1903-01-20  46789746      F24C   1/02
1  1113768         US  1914-10-13  47666487    A61M   1/0005
2  1113768         US  1914-10-13  47666487     F04B  53/164
3  1216480         US  1917-02-20  47845187      H04M   1/02
4   353089         US  1886-11-23  46162467     A63B  21/062


1747713

In [21]:
print 'upper bound on number of gained patents from CPC',
print len(USdf_needed.appln_id.value_counts())
# num of extra patents getting from CPC
print 'number of codes with only a single patent:',
print USdf_needed.groupby('appln_id').size().value_counts()[:1]
print 1085145/float(1346375)

upper bound on number of gained patents from CPC 1346375
number of codes with only a single patent: 1    1085145
dtype: int64
0.805975304057


In [22]:
# truncate on '/'
df.loc[:,'ipc_class_symbol'] = df.loc[:,'ipc_class_symbol'].map(lambda x: x.split('/')[0])
USdf_needed.loc[:,'cpc_class_symbol'] = USdf_needed.loc[:,'cpc_class_symbol'].map(lambda x: x.split('/')[0])
print df.head()
print USdf_needed.head()
print '\n'
print len(df)
print df.ipc_class_symbol.map(len).value_counts()
print len(USdf_needed)
print USdf_needed.cpc_class_symbol.map(len).value_counts()

  publn_nr appln_auth  publn_date  appln_id ipc_class_symbol
0  PP19626         US  2009-01-20  55754593         A01H   5
1  PP19627         US  2009-01-20  52899962         A01H   5
2  PP19628         US  2009-01-20  55754594         A01H   5
3  PP19629         US  2009-01-20  55754595         A01H   5
4  PP19630         US  2009-01-20  55318791         A01H   5
  publn_nr appln_auth  publn_date  appln_id cpc_class_symbol
0   718793         US  1903-01-20  46789746         F24C   1
1  1113768         US  1914-10-13  47666487         A61M   1
2  1113768         US  1914-10-13  47666487         F04B  53
3  1216480         US  1917-02-20  47845187         H04M   1
4   353089         US  1886-11-23  46162467         A63B  21


45545566
8    45116185
4      429381
Name: ipc_class_symbol, dtype: int64
1747713
8    1747713
Name: cpc_class_symbol, dtype: int64


//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:508: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [23]:
# split into code groups and only code subclasses by filtering on ipc contain whitespace
df_ipc_prime = df[df.ipc_class_symbol.str.contains(' ')]
print df_ipc_prime.head()
print len(df_ipc_prime)
df_ipc_4 = df[~df.ipc_class_symbol.str.contains(' ')]
print df_ipc_4.head()
print len(df_ipc_4)
print '\n'
print df_ipc_prime.ipc_class_symbol.map(len).value_counts()
print df_ipc_4.ipc_class_symbol.map(len).value_counts()
# checks all good

  publn_nr appln_auth  publn_date  appln_id ipc_class_symbol
0  PP19626         US  2009-01-20  55754593         A01H   5
1  PP19627         US  2009-01-20  52899962         A01H   5
2  PP19628         US  2009-01-20  55754594         A01H   5
3  PP19629         US  2009-01-20  55754595         A01H   5
4  PP19630         US  2009-01-20  55318791         A01H   5
45116185
     publn_nr appln_auth  publn_date  appln_id ipc_class_symbol
645   7479062         US  2009-01-20  46780856             G06F
779   7479168         US  2009-01-20  49994785             C07C
782   7479168         US  2009-01-20  49994785             C10G
1497  7479686         US  2009-01-20  49344017             H01L
1941  7480056         US  2009-01-20  53869339             G02F
429381


8    45116185
Name: ipc_class_symbol, dtype: int64
4    429381
Name: ipc_class_symbol, dtype: int64


In [24]:
print 45109055 + 429396 

45538451


In [25]:
# number of patents lost 

In [26]:
c = 0
for code in df_ipc_prime.ipc_class_symbol:
    if code[5] != ' ':
        c = c+1
print c

946933


In [27]:
# Split CPCs into those needing to be munged and those not
USdf_cpc_spaces = USdf_needed[USdf_needed.cpc_class_symbol.str.contains(' ')]
print USdf_cpc_spaces.head()
print len(USdf_cpc_spaces)
print '\n'
USdf_cpc_only = USdf_needed[~USdf_needed.cpc_class_symbol.str.contains(' ')]
print USdf_cpc_only.head()
print len(USdf_cpc_only)

  publn_nr appln_auth  publn_date  appln_id cpc_class_symbol
0   718793         US  1903-01-20  46789746         F24C   1
1  1113768         US  1914-10-13  47666487         A61M   1
2  1113768         US  1914-10-13  47666487         F04B  53
3  1216480         US  1917-02-20  47845187         H04M   1
4   353089         US  1886-11-23  46162467         A63B  21
1667829


   publn_nr appln_auth  publn_date  appln_id cpc_class_symbol
50    18021         US  1857-08-18  45415012         A61F2002
57    22575         US  1859-01-11  45427105         A61F2002
74    26378         US  1859-12-06  45437310         A61F2002
84    27264         US  1860-02-28  45439660         C04B2235
85    27264         US  1860-02-28  45439660         C04B2235
79884


In [28]:
print 1667829 + 79884
print 1747713 

1747713
1747713


In [29]:
# Check the not mungeneed all exist in ipc, should get no results if all matching
USdf_cpc_spaces_no_match = USdf_cpc_spaces[~USdf_cpc_spaces.cpc_class_symbol.isin(df_ipc_prime.ipc_class_symbol)]
USdf_cpc_spaces_with_match = USdf_cpc_spaces[USdf_cpc_spaces.cpc_class_symbol.isin(df_ipc_prime.ipc_class_symbol)]
print 'number of CPC assignations with no match in IPC:',
print len(USdf_cpc_spaces_no_match) + len(USdf_cpc_only)
print '\n'
#print USdf_cpc_spaces_no_match.cpc_class_symbol.drop_duplicates()

print 'number of CPC assignations with a match in IPC:',
print len(USdf_cpc_spaces_with_match)

number of CPC assignations with no match in IPC: 326713


number of CPC assignations with a match in IPC: 1421000


In [30]:
print 1667829
print 1421000 + 246829

1667829
1667829


## 'CPC only' codes cannot be reliably munged
(Could do this using the concordance files but not worth the time)
See CPC guide page 8 for examples of ambiguous codes
## CPC to IPC as far is it could go

In [31]:
# make the matching CPCs apps data ready to be appended
US_extra_fin =  USdf_cpc_spaces_with_match.copy()
print len(US_extra_fin)
print len(USdf_cpc_spaces_with_match)
US_extra_fin.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True)
US_extra_fin.head()

1421000
1421000


,publn_nr,appln_auth,publn_date,appln_id,ipc_class_symbol
0,718793,US,1903-01-20,46789746,F24C 1
1,1113768,US,1914-10-13,47666487,A61M 1
2,1113768,US,1914-10-13,47666487,F04B 53
3,1216480,US,1917-02-20,47845187,H04M 1
4,353089,US,1886-11-23,46162467,A63B 21


In [32]:
# append the prime IPC with the prime US, making final data set for usage
df_final_full = df_ipc_prime.append(US_extra_fin, ignore_index=True)
print df_final_full.head()
print len(df_final_full)
print len(US_extra_fin)+len(df_ipc_prime)

  publn_nr appln_auth  publn_date  appln_id ipc_class_symbol
0  PP19626         US  2009-01-20  55754593         A01H   5
1  PP19627         US  2009-01-20  52899962         A01H   5
2  PP19628         US  2009-01-20  55754594         A01H   5
3  PP19629         US  2009-01-20  55754595         A01H   5
4  PP19630         US  2009-01-20  55318791         A01H   5
46537185
46537185


In [33]:
df_final_full

,publn_nr,appln_auth,publn_date,appln_id,ipc_class_symbol
0,PP19626,US,2009-01-20,55754593,A01H 5
1,PP19627,US,2009-01-20,52899962,A01H 5
2,PP19628,US,2009-01-20,55754594,A01H 5
3,PP19629,US,2009-01-20,55754595,A01H 5
4,PP19630,US,2009-01-20,55318791,A01H 5
5,PP19631,US,2009-01-20,55754596,A01H 5
6,PP19632,US,2009-01-20,52909593,A01H 5
7,PP19633,US,2009-01-20,55754597,A01H 5
8,PP19634,US,2009-01-20,55754598,A01H 5
9,PP19635,US,2009-01-20,55754599,A01H 5


# Now filter off single code patents (as they can't result in links)

In [34]:
app_code_num = df_final_full.groupby('appln_id').size()
app_code_num.name = 'num'
app_code_num

appln_id
10000006    1
10000009    1
10000010    3
10000011    1
10000021    3
10000024    2
10000063    2
10000083    1
10000095    3
10000097    1
10000109    2
10000111    1
10000134    1
10000141    1
10000158    1
10000159    1
10000160    1
10000168    1
10000174    2
10000191    1
10000211    2
10000214    1
10000215    2
10000227    1
10000229    1
10000236    1
10000261    2
10000285    8
10000286    1
10000291    1
           ..
9999775     2
9999783     5
9999791     1
9999795     4
9999796     2
9999797     1
9999803     2
9999807     1
9999812     1
9999813     7
9999832     4
9999847     1
9999848     3
9999850     2
9999855     2
9999861     1
9999875     2
9999877     2
9999881     1
9999894     7
9999898     4
9999905     2
9999913     3
9999916     3
9999921     1
9999932     1
9999936     1
9999970     1
9999973     2
9999981     1
Name: num, dtype: int64

In [35]:
print len(app_code_num[app_code_num == 1])
print len(app_code_num[app_code_num == 1])/float(len(df_final_full)) 
# this is farction of codes lost not patents lost...

12223310
0.262656840976


In [36]:
app_code_num = app_code_num.reset_index()
print len(app_code_num[app_code_num.num == 1])/float(len(df_final_full.drop_duplicates('appln_id')))
# this is fraction patents lost by filter on single codes

0.536023078008


In [37]:
# pre check num of codes should be in full dataset
print  len(df_final_full) - len(app_code_num[app_code_num.num == 1])

34313875


In [38]:
len(df_final_full[~df_final_full.appln_id.isin(app_code_num[app_code_num.num == 1].appln_id)])
# check good

34313875

In [39]:
# get list data for single code patents before cutting them out data frame (for completeness checks later)
rej_single_code = df_final_full[df_final_full.appln_id.isin(app_code_num[app_code_num.num == 1].appln_id)]
print len(rej_single_code)

12223310


In [40]:
# save this set for later
#pk.dump(rej_single_code, open('pickles_and_munged_csv/single_code_pats_v1.pk','wb'))
#rej_single_code.to_csv(path_or_buf='pickles_and_munged_csv/single_code_pats_v1.csv', index=False)

In [41]:
rej_single_code.head()

,publn_nr,appln_auth,publn_date,appln_id,ipc_class_symbol
0,PP19626,US,2009-01-20,55754593,A01H 5
1,PP19627,US,2009-01-20,52899962,A01H 5
2,PP19628,US,2009-01-20,55754594,A01H 5
3,PP19629,US,2009-01-20,55754595,A01H 5
4,PP19630,US,2009-01-20,55318791,A01H 5


In [42]:
df_final_full = df_final_full[~df_final_full.appln_id.isin(app_code_num[app_code_num.num == 1].appln_id)]

In [43]:
#final check
(df_final_full.groupby('appln_id').size() == 1).sum()

0

In [44]:
# how many patents with multiply codes after deduplicating codes (i.e losing self links)
print len(df_final_full.drop_duplicates(['appln_id','ipc_class_symbol']))
print len(df_final_full)
print len(df_final_full) - len(df_final_full.drop_duplicates(['appln_id','ipc_class_symbol']))
print (df_final_full.drop_duplicates(['appln_id','ipc_class_symbol']).groupby('appln_id').size() ==1).sum()
print (df_final_full.drop_duplicates(['appln_id','ipc_class_symbol']).groupby('appln_id').size() !=1).sum()
print len(df_final_full.drop_duplicates('appln_id'))
print ((df_final_full.drop_duplicates(['appln_id','ipc_class_symbol']).groupby('appln_id').size() ==1).sum()
/float(len(df_final_full.drop_duplicates('appln_id'))))

25554949
34313875
8758926
2045365
8535026
10580391
0.193316579699


In [45]:
print ((df_final_full.drop_duplicates(['appln_id','ipc_class_symbol']).groupby('appln_id').size() ==1).sum()
/float(len(df_final_full)))

0.0596075202815


In [46]:
34313875 - 25554949

8758926

In [47]:
df_final_full

,publn_nr,appln_auth,publn_date,appln_id,ipc_class_symbol
32,RE40623,US,2009-01-20,55754591,G06K 19
33,RE40623,US,2009-01-20,55754591,H01L 21
34,RE40623,US,2009-01-20,55754591,H01L 23
35,RE40623,US,2009-01-20,55754591,H01L 23
38,7478442,US,2009-01-20,46977285,A47K 3
39,7478442,US,2009-01-20,46977285,E04H 4
40,7478446,US,2009-01-20,54055206,A61G 7
41,7478446,US,2009-01-20,54055206,A61G 7
42,7478449,US,2009-01-20,51002082,E01D 1
43,7478449,US,2009-01-20,51002082,E01D 15


In [48]:
#%time pk.dump(df_final_full, open('pickles_and_munged_csv/final_data_v3.pk', 'wb'))
#%time df_final_full.to_csv(path_or_buf='pickles_and_munged_csv/final_data_v3.csv', index=False)

# Country Selection Criteria

In [49]:
#Munge dates
#Date to years function - works on application date and publication date in one
def date_to_year(df):
    if 'appln_filing_date' in df.columns:
        df.appln_filing_date = df.appln_filing_date.map(lambda x:int(x[:4]))
        df.rename(columns={'appln_filing_date':'app_year'}, inplace=True)
    if 'publn_date'in df.columns:
        df.publn_date = df.publn_date.map(lambda x:int(x[:4]))
        df.rename(columns={'publn_date':'pub_year'}, inplace=True)
    return

In [50]:
df_final_app_selection = df_final_full.copy()
df_final_app_selection = df_final_app_selection.drop('ipc_class_symbol', axis=1)
df_final_app_selection = df_final_app_selection.drop('publn_nr', axis=1)
date_to_year(df_final_app_selection)
df_final_app_selection = df_final_app_selection.drop_duplicates()
df_final_app_selection

,appln_auth,pub_year,appln_id
32,US,2009,55754591
38,US,2009,46977285
40,US,2009,54055206
42,US,2009,51002082
46,US,2009,53487594
50,US,2009,53439703
55,US,2009,48888602
57,US,2009,55386992
60,US,2009,50823771
67,US,2009,51282821


In [51]:
final_grp = df_final_app_selection.groupby(['appln_auth','pub_year'])

In [52]:
final_sizes = final_grp.size()
final_sizes.head()

appln_auth  pub_year
            1983        2
            1984        3
            1985        3
            1986        4
            1987        4
dtype: int64

In [53]:
print final_sizes.index.get_level_values('pub_year') < 1920
print final_sizes.index.get_level_values('pub_year')

[False False False ..., False False False]
Int64Index([1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
            ...
            1982, 1984, 1986, 1987, 1989, 1990, 1982, 1986, 1989, 1990],
           dtype='int64', name=u'pub_year', length=3158)


In [54]:
top10 = final_sizes.sum(level=0).sort_values(ascending=False)[:11].index.values
top20 = final_sizes.sum(level=0).sort_values(ascending=False)[:21].index.values
print top10
print top20
#pk.dump(top10, open('pickles_and_munged_csv/final_top10_v1.pk','wb'))

['JP' 'CN' 'US' 'DE' 'KR' 'FR' 'SU' 'GB' 'RU' 'TW' 'CH']
['JP' 'CN' 'US' 'DE' 'KR' 'FR' 'SU' 'GB' 'RU' 'TW' 'CH' 'ES' 'IT' 'PL' 'AT'
 'CA' 'DD' 'AU' 'NL' 'SE' 'FI']


In [55]:
pre1950_sizes_by_yr = final_sizes.iloc[final_sizes.index.get_level_values('pub_year') < 1950
pre1950_tot_sizes = final_sizes.iloc[final_sizes.index.get_level_values('pub_year') < 1950].sum(level=0).sort_values(ascending=False)
print pre1950_sizes_by_yr
print pre1950_tot_sizes

SyntaxError: invalid syntax (<ipython-input-55-1786bc7247e2>, line 2)

In [101]:
top10_pre1930 = final_sizes.iloc[final_sizes.index.get_level_values('pub_year') < 1930].sum(level=0).sort_values(ascending=False)[0:11]
top20_pre1930 = final_sizes.iloc[final_sizes.index.get_level_values('pub_year') < 1930].sum(level=0).sort_values(ascending=False)[0:21]
print top10_pre1930
print top20_pre1930



appln_auth
US    135752
FR     24404
GB     22998
DE     17706
SU      2608
CH       643
NL       146
BE        70
AT        64
DK        13
SE         2
dtype: int64
appln_auth
US    135752
FR     24404
GB     22998
DE     17706
SU      2608
CH       643
NL       146
BE        70
AT        64
DK        13
SE         2
CA         2
JP         1
dtype: int64


In [93]:
print final_sizes.iloc[final_sizes.index.get_level_values('pub_year') < 2000].sum(level=0).sort_values(ascending=False)

IndexError: index 2819816 is out of bounds for axis 1 with size 84

# Checking Completeness (seeing where the loses of data occur)

## Want to look at fraction of patents that have a missing code per auth per year

In [48]:
# DONT NEED THIS, USE THE PICKLE OF THE FULL FULL NUMBERS IN PATSTAT
# Still good for checking total code count is conserved, and checking data loss at each step
# Make set for no filter on codes 
USdf_needed.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True)

full_no_filter_df = df.append(USdf_needed, ignore_index=True)

//anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [49]:
# NEED TO INCLUDE PATS WITH SINGLE CODES HERE TOO, lets call it rej_single_code
# Make set of rejected codes
USdf_cpc_only.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True)
USdf_cpc_spaces_no_match.rename(columns={'cpc_class_symbol':'ipc_class_symbol'}, inplace=True)
reject_codes = df_ipc_4.append([USdf_cpc_only,
                                USdf_cpc_spaces_no_match,
                                rej_single_code],
                               ignore_index=True)

In [50]:
print len(full_no_filter_df)
print len(df_final_still_EP)
print len(reject_codes)

47293279
34313875
12979404


In [51]:
print len(full_no_filter_df)
print len(df_final_still_EP) + len(reject_codes)
print len(reject_codes)
print len(df_ipc_4) + len(USdf_cpc_only) + len(USdf_cpc_spaces_no_match) + len(rej_single_code) 
# checks, something weird was going with append, think there was a command that assigned to a copy of a df rather
    #than a view (a slice of the df not copied) of the df, (i.e the assignation actually left the da) 

47293279
47293279
12979404
12979404


### Drop classifcation columns, then remove duplicates on appln_id so each patent is in each dataframe once

In [52]:
full_no_filter_df.drop('ipc_class_symbol', axis=1, inplace=True)
df_final_still_EP.drop('ipc_class_symbol', axis=1, inplace=True)
reject_codes.drop('ipc_class_symbol', axis=1, inplace=True)
rej_single_code.drop('ipc_class_symbol', axis=1, inplace=True)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [53]:
full_no_filter_df.drop_duplicates()

,publn_nr,appln_auth,publn_date,appln_id
0,PP19626,US,2009-01-20,55754593
1,PP19627,US,2009-01-20,52899962
2,PP19628,US,2009-01-20,55754594
3,PP19629,US,2009-01-20,55754595
4,PP19630,US,2009-01-20,55318791
5,PP19631,US,2009-01-20,55754596
6,PP19632,US,2009-01-20,52909593
7,PP19633,US,2009-01-20,55754597
8,PP19634,US,2009-01-20,55754598
9,PP19635,US,2009-01-20,55754599


In [54]:
full_no_filter_df.drop_duplicates('appln_id')
# shows one to one matching between publn_nr and appln_id

,publn_nr,appln_auth,publn_date,appln_id
0,PP19626,US,2009-01-20,55754593
1,PP19627,US,2009-01-20,52899962
2,PP19628,US,2009-01-20,55754594
3,PP19629,US,2009-01-20,55754595
4,PP19630,US,2009-01-20,55318791
5,PP19631,US,2009-01-20,55754596
6,PP19632,US,2009-01-20,52909593
7,PP19633,US,2009-01-20,55754597
8,PP19634,US,2009-01-20,55754598
9,PP19635,US,2009-01-20,55754599


In [55]:
full_no_filter_df.drop_duplicates(inplace=True)
df_final_still_EP.drop_duplicates(inplace=True)
reject_codes.drop_duplicates(inplace=True)
rej_single_code.drop_duplicates(inplace=True)

//anaconda/lib/python2.7/site-packages/pandas/util/decorators.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [56]:
print len(full_no_filter_df)
print full_no_filter_df.appln_id.isin(df_final_still_EP.appln_id).value_counts()
print full_no_filter_df.appln_id.isin(reject_codes.appln_id).value_counts()
print full_no_filter_df.appln_id.isin(rej_single_code.appln_id).value_counts()
print df_final_still_EP.appln_id.isin(full_no_filter_df.appln_id).value_counts()
print reject_codes.appln_id.isin(full_no_filter_df.appln_id).value_counts()
print rej_single_code.appln_id.isin(full_no_filter_df.appln_id).value_counts()
# sanity checks

23136784
False    12550733
True     10586051
Name: appln_id, dtype: int64
True     12623448
False    10513336
Name: appln_id, dtype: int64
True     12223310
False    10913474
Name: appln_id, dtype: int64
True    10586051
Name: appln_id, dtype: int64
True    12623448
Name: appln_id, dtype: int64
True    12223310
Name: appln_id, dtype: int64


In [57]:
date_to_year(full_no_filter_df); print full_no_filter_df.head()
date_to_year(df_final_still_EP); print df_final_still_EP.head()
date_to_year(reject_codes); print reject_codes.head()
date_to_year(rej_single_code); print rej_single_code.head()

  publn_nr appln_auth  pub_year  appln_id
0  PP19626         US      2009  55754593
1  PP19627         US      2009  52899962
2  PP19628         US      2009  55754594
3  PP19629         US      2009  55754595
4  PP19630         US      2009  55318791
   publn_nr appln_auth  pub_year  appln_id
32  RE40623         US      2009  55754591
38  7478442         US      2009  46977285
40  7478446         US      2009  54055206
42  7478449         US      2009  51002082
46  7478451         US      2009  53487594
  publn_nr appln_auth  pub_year  appln_id
0  7479062         US      2009  46780856
1  7479168         US      2009  49994785
3  7479686         US      2009  49344017
4  7480056         US      2009  53869339
5  7480233         US      2009  47907909
  publn_nr appln_auth  pub_year  appln_id
0  PP19626         US      2009  55754593
1  PP19627         US      2009  52899962
2  PP19628         US      2009  55754594
3  PP19629         US      2009  55754595
4  PP19630         US      2

//anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [58]:
df_auth_years_nums = full_no_filter_df.groupby(['appln_auth', 'pub_year'])

In [59]:
full_no_filter_df['is_missing'] = ~full_no_filter_df.appln_id.isin(df_final_still_EP.appln_id)
full_no_filter_df

,publn_nr,appln_auth,pub_year,appln_id,is_missing
0,PP19626,US,2009,55754593,True
1,PP19627,US,2009,52899962,True
2,PP19628,US,2009,55754594,True
3,PP19629,US,2009,55754595,True
4,PP19630,US,2009,55318791,True
5,PP19631,US,2009,55754596,True
6,PP19632,US,2009,52909593,True
7,PP19633,US,2009,55754597,True
8,PP19634,US,2009,55754598,True
9,PP19635,US,2009,55754599,True


In [60]:
# check
full_no_filter_df.is_missing.value_counts()

True     12550733
False    10586051
Name: is_missing, dtype: int64

In [61]:
full_no_filter_df['is_missing_due_single_c'] = full_no_filter_df.appln_id.isin(rej_single_code.appln_id)
full_no_filter_df

,publn_nr,appln_auth,pub_year,appln_id,is_missing,is_missing_due_single_c
0,PP19626,US,2009,55754593,True,True
1,PP19627,US,2009,52899962,True,True
2,PP19628,US,2009,55754594,True,True
3,PP19629,US,2009,55754595,True,True
4,PP19630,US,2009,55318791,True,True
5,PP19631,US,2009,55754596,True,True
6,PP19632,US,2009,52909593,True,True
7,PP19633,US,2009,55754597,True,True
8,PP19634,US,2009,55754598,True,True
9,PP19635,US,2009,55754599,True,True


In [62]:
full_no_filter_df['is_incomplete_inc_missing'] = full_no_filter_df.appln_id.isin(reject_codes.appln_id)
full_no_filter_df

,publn_nr,appln_auth,pub_year,appln_id,is_missing,is_missing_due_single_c,is_incomplete_inc_missing
0,PP19626,US,2009,55754593,True,True,True
1,PP19627,US,2009,52899962,True,True,True
2,PP19628,US,2009,55754594,True,True,True
3,PP19629,US,2009,55754595,True,True,True
4,PP19630,US,2009,55318791,True,True,True
5,PP19631,US,2009,55754596,True,True,True
6,PP19632,US,2009,52909593,True,True,True
7,PP19633,US,2009,55754597,True,True,True
8,PP19634,US,2009,55754598,True,True,True
9,PP19635,US,2009,55754599,True,True,True


In [63]:
# check
full_no_filter_df.is_incomplete_inc_missing.value_counts()

True     12623448
False    10513336
Name: is_incomplete_inc_missing, dtype: int64

In [64]:
full_no_filter_df.drop('publn_nr', axis=1, inplace=True); full_no_filter_df.head()

,appln_auth,pub_year,appln_id,is_missing,is_missing_due_single_c,is_incomplete_inc_missing
0,US,2009,55754593,True,True,True
1,US,2009,52899962,True,True,True
2,US,2009,55754594,True,True,True
3,US,2009,55754595,True,True,True
4,US,2009,55318791,True,True,True


In [65]:
full_no_filter_df.drop('appln_id', axis=1, inplace=True); full_no_filter_df.head()

,appln_auth,pub_year,is_missing,is_missing_due_single_c,is_incomplete_inc_missing
0,US,2009,True,True,True
1,US,2009,True,True,True
2,US,2009,True,True,True
3,US,2009,True,True,True
4,US,2009,True,True,True


# Grouping by Auth and Year

In [66]:
full_grp = full_no_filter_df.groupby(['appln_auth', 'pub_year'])

## We want t the full number that exist in PATSTAT, thus we can measure our total loss of number of patents from the total in PATSTAT (granted only prior) to those with acceptable codes
Not just the total that have classifacitons within PATSTAT (represented by `full_no_filter_df`)


So grab that from a pickle from 'All Granted Only Priors - Completeness' notebook

In [67]:
s_au_year_tot_num = pk.load(open('pickles_and_munged_csv/s_au_year_num.pk','rb')) 
s_au_year_tot_num.name = 'tot_num'

In [68]:
s_app_num = full_grp.size()
s_app_num.name = 'num_in_class_tables' ; print s_app_num.head()
s_missing_num = full_grp.is_missing.sum()
s_missing_num.name = 'num_missing_post_filter'; print s_missing_num.head()
s_incom_num = full_grp.is_incomplete_inc_missing.sum()
s_incom_num.name = 'num_with_incom_or_miss_post_filter' ; print s_incom_num.head()
s_miss_due_sin_num = full_grp.is_missing_due_single_c.sum()
s_miss_due_sin_num.name = 'num_miss_due_sin' ; print s_miss_due_sin_num.head()

appln_auth  pub_year
            1983        2
            1984        3
            1985        6
            1986        7
            1987        7
Name: num_in_class_tables, dtype: int64
appln_auth  pub_year
            1983        0
            1984        0
            1985        3
            1986        3
            1987        3
Name: num_missing_post_filter, dtype: float64
appln_auth  pub_year
            1983        0
            1984        0
            1985        3
            1986        3
            1987        3
Name: num_with_incom_or_miss_post_filter, dtype: float64
appln_auth  pub_year
            1983        0
            1984        0
            1985        3
            1986        3
            1987        3
Name: num_miss_due_sin, dtype: float64


In [69]:
# checks 
print len(s_au_year_tot_num)
print len(s_app_num)
test_df_tot_nums = pd.concat([s_au_year_tot_num, s_app_num], axis=1); print test_df_tot_nums
test_df_tot_nums[pd.isnull(test_df_tot_nums.num)]

3488
3445
                     tot_num  num_in_class_tables
appln_auth pub_year                              
           1983          NaN                    2
           1984          NaN                    3
           1985          NaN                    6
           1986          NaN                    7
           1987          NaN                    7
           1988          NaN                   12
           1989          NaN                    6
           1990          NaN                    9
           1991          NaN                    5
           1992          NaN                    4
           1993          NaN                    4
           1994          NaN                    5
           1995          NaN                    5
           1996          NaN                   11
           1997          NaN                    7
           1998          NaN                    3
           1999          NaN                    2
           2000          NaN            

AttributeError: 'DataFrame' object has no attribute 'num'

In [ ]:
test_df_tot_nums.fillna(0)
test_df_tot_nums[test_df_tot_nums.tot_num < test_df_tot_nums.num_in_class_tables]
# all checks pass

In [ ]:
tot_mis_inc_stats = pd.concat([s_au_year_tot_num, s_app_num, s_missing_num, s_miss_due_sin_num, s_incom_num], axis=1)

In [ ]:
# fill missings with totally num for totally missing (just for auth 'BY')
tot_mis_inc_stats.loc[:,'num_in_class_tables'] =  tot_mis_inc_stats.num_in_class_tables.fillna(0) 
tot_mis_inc_stats.loc[:,'num_missing_post_filter'] =  tot_mis_inc_stats.num_missing_post_filter.fillna(0) 
tot_mis_inc_stats.loc[:,'num_with_incom_or_miss_post_filter'] =  tot_mis_inc_stats.num_with_incom_or_miss_post_filter.fillna(0) 
tot_mis_inc_stats.loc[:,'num_miss_due_sin'] =  tot_mis_inc_stats.num_miss_due_sin.fillna(0) 
tot_mis_inc_stats.loc['BY']

In [ ]:
# Want totals missing, right last two cols only count missing from the num_in_class_tables
# add to last two cols the difference between first two
tot_mis_inc_stats.num_missing_post_filter =  ((tot_mis_inc_stats.tot_num
                                                    - tot_mis_inc_stats.num_in_class_tables)
                                                   + tot_mis_inc_stats.num_missing_post_filter)
tot_mis_inc_stats.num_with_incom_or_miss_post_filter =  ((tot_mis_inc_stats.tot_num
                                                    - tot_mis_inc_stats.num_in_class_tables)
                                                   + tot_mis_inc_stats.num_with_incom_or_miss_post_filter)
tot_mis_inc_stats.num_miss_due_sin =  ((tot_mis_inc_stats.tot_num
                                                    - tot_mis_inc_stats.num_in_class_tables)
                                                   + tot_mis_inc_stats.num_miss_due_sin)
tot_mis_inc_stats

In [ ]:
# Process into new dataframe of fractional quantities ready to plot, for zero entries used 1

tot_mis_frac = tot_mis_inc_stats.div(tot_mis_inc_stats['tot_num'], axis='index')
tot_mis_frac.rename(columns={key: key.replace("num", 'frac') for key in tot_mis_frac.columns}, inplace=True)
tot_mis_frac

In [ ]:
# Pivot on authorieties (need to unindex it first)
tot_mis_frac_pv = tot_mis_frac.reset_index(level=0).pivot(columns='appln_auth')
tot_mis_frac_pv = tot_mis_frac_pv.fillna(0) # made value for year with no patents equal to 0 i.e non are missing or incomplete
tot_mis_frac_pv

# Just By Authority (not year also)

In [ ]:
# load authority sizings etc
auth_by_size = pk.load(open('pickles_and_munged_csv/auth_by_size.pk', 'rb'))
eu_auths = pk.load(open('pickles_and_munged_csv/eu_auths.pk', 'rb'))
auth_sizes = pk.load(open('pickles_and_munged_csv/auth_sizes.pk', 'rb'))

In [ ]:
full_auths_grp = full_no_filter_df.drop('pub_year', axis=1).groupby('appln_auth')

In [ ]:
s_auths_num = full_auths_grp.size()
s_auths_num.name = 'num_in_class_tables' ; print s_auths_num.head()
s_auths_missing_num = full_auths_grp.is_missing.sum()
s_auths_missing_num.name = 'num_missing_post_filter'; print s_auths_missing_num.head()
s_auths_incom_num = full_auths_grp.is_incomplete_inc_missing.sum()
s_auths_incom_num.name = 'num_with_incom_or_miss_post_filter' ; print s_auths_incom_num.head()
s_auths_miss_due_sin_num = full_auths_grp.is_missing_due_single_c.sum()
s_auths_miss_due_sin_num.name = 'num_miss_due_sin' ; print s_miss_due_sin_num.head()

In [ ]:
auth_sizes.name = 'tot_num'
df_auths_mis_inc_stats = pd.concat([auth_sizes, s_auths_num, s_auths_missing_num, s_auths_miss_due_sin_num, s_auths_incom_num], axis=1)
# fill missings with totally num for totally missing (just for auth 'BY')
df_auths_mis_inc_stats.loc[:,'num_in_class_tables'] =  df_auths_mis_inc_stats.num_in_class_tables.fillna(0) 
df_auths_mis_inc_stats.loc[:,'num_missing_post_filter'] =  df_auths_mis_inc_stats.num_missing_post_filter.fillna(0) 
df_auths_mis_inc_stats.loc[:,'num_with_incom_or_miss_post_filter'] =  df_auths_mis_inc_stats.num_with_incom_or_miss_post_filter.fillna(0) 
df_auths_mis_inc_stats.loc[:,'num_miss_due_sin'] =  df_auths_mis_inc_stats.num_miss_due_sin.fillna(0) 
df_auths_mis_inc_stats

In [ ]:
# Want totals missing, right last two cols only count missing from the num_in_class_tables
# add to last two cols the difference between first two
df_auths_mis_inc_stats.num_missing_post_filter =  ((df_auths_mis_inc_stats.tot_num
                                                    - df_auths_mis_inc_stats.num_in_class_tables)
                                                   + df_auths_mis_inc_stats.num_missing_post_filter)
df_auths_mis_inc_stats.num_with_incom_or_miss_post_filter =  ((df_auths_mis_inc_stats.tot_num
                                                    - df_auths_mis_inc_stats.num_in_class_tables)
                                                   + df_auths_mis_inc_stats.num_with_incom_or_miss_post_filter)
df_auths_mis_inc_stats.num_miss_due_sin =  ((df_auths_mis_inc_stats.tot_num
                                                    - df_auths_mis_inc_stats.num_in_class_tables)
                                                   + df_auths_mis_inc_stats.num_miss_due_sin)
df_auths_mis_inc_stats

In [ ]:
# Process into new dataframe of fractional quantities ready to plot, for zero entries used 1

df_auths_mis_inc_frac = df_auths_mis_inc_stats.div(df_auths_mis_inc_stats['tot_num'], axis='index')
#df_auths_mis_inc_frac.fillna(0, inplace=True) 
df_auths_mis_inc_frac.rename(columns={key: key.replace("num", 'frac') for key in df_auths_mis_inc_frac.columns},
                             inplace=True)
df_auths_mis_inc_frac

## Ranking / Plotting

In [ ]:
# Ranking by applications not found in classification tables 
df_auths_mis_inc_frac.sort_values(by='frac_missing_post_filter', ascending=False)

In [ ]:
N = len(df_auths_mis_inc_frac)
fs = np.array(range(N))
xs = df_auths_mis_inc_frac.sort_values(by='frac_missing_post_filter',
                                       ascending=True).frac_missing_post_filter.values
plt.plot(fs, xs)
#plt.margins(0.1, 0.1)

ax = plt.gca()
#ax.set_xscale('log')
#ax.set_yscale('log')
#plt.savefig('figures/all_grant_auth_dists_v1.pdf')

print df_auths_mis_inc_frac.sort_values(by='frac_missing_post_filter',
                                       ascending=True).frac_missing_post_filter

In [ ]:
N = len(df_auths_mis_inc_frac)
fs = np.array(range(N))
xs = df_auths_mis_inc_frac.sort_values(by='frac_with_incom_or_miss_post_filter',
                                       ascending=True).frac_with_incom_or_miss_post_filter.values
plt.plot(fs, xs)
#plt.margins(0.1, 0.1)

ax = plt.gca()
#ax.set_xscale('log')
#ax.set_yscale('log')
#plt.savefig('figures/all_grant_auth_dists_v1.pdf')

In [ ]:
df_auths_mis_inc_frac.sort_values(by='frac_with_incom_or_miss_post_filter', ascending=False)